In [1]:
import os
import pandas as pd; pd.set_option('display.max_columns', None) 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pickle
from IPython.display import clear_output as co
from random import random
import warnings; # ...

import sklearn.preprocessing
import sklearn.model_selection
import sklearn.metrics
import sklearn.utils

from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.statespace.varmax import VARMAX

#!pip install pmdarima
from pmdarima.arima import auto_arima
#!pip install TBATS
from tbats import TBATS

In [2]:
le = sklearn.preprocessing.LabelEncoder
#regressors = dict(sklearn.utils.all_estimators('regressor'))
metrics = sklearn.metrics._regression
mae = metrics.mean_absolute_error
mse = metrics.mean_squared_error
mape = metrics.mean_absolute_percentage_error
r2 = metrics.r2_score
me = metrics.max_error
medae = metrics.median_absolute_error
evs = metrics.explained_variance_score
mpd = metrics.mean_poisson_deviance
mgd = metrics.mean_gamma_deviance
def wape(ye, yhat):
    return np.abs(ye-yhat).sum()/ye.sum()
def e(ye, yhat):
    return np.abs(yhat-ye).sum()
def estd(ye, yhat):
    return np.abs(yhat-ye).std()

### Declarando Funções

In [3]:
def load_series(path='series/'):
    filenames = os.listdir(path)
    series, cnt, n_files = {}, 0, len(filenames)
    freq = range(1, n_files, 10)
    for filename in filenames:
        cnt+=1
        if cnt in freq: print(f'Files loaded: {cnt}/{n_files}'); co(wait=True)
        series[filename[:-4]] = pd.read_csv(path+filename, index_col=0)
    print(f'Done! Loaded {n_files} files.')
    return series

## Return random serie's city and product-type
def sample_serie():
    keys = np.random.choice(list(series.keys())).split('-')
    return keys[0], keys[1:]
    
def clean_series(series, train_min=1, drop_start_zeros=True):
    clean_series = {}
    test_empty = []
    excluded_test_index = []
    n_total = len(series)
    n_values_total = 0
    n_values_test = 0
    n_values_train = 0
    n_series = 0
    n_series_empty = 0
    n_series_test_empty = 0
    n_series_train_min_empty = 0
    n_values = 0
    n_test_empty_total = 0
    n_train_empty_total = 0
    n_non_empty_values = 0
    n_train_zeros = 0

    cnt, freq = 0, range(1, n_total, 10)
    for key in list(series.keys()):
        cnt+=1
        if cnt in freq: print(f'Cleaned series: {cnt}/{n_total}'); co(wait=True)

        msk = series[key]['year'].isin(['01/01/2016', '01/01/2017'])
        test = series[key][msk]
        train = series[key][msk==False]

        n = len(series[key])
        n_empty = series[key][target].isna().sum()
        n_test_empty = test[target].isna().sum()
        n_train_empty = train[target].isna().sum()

        cond1 = n_test_empty == len(test)
        cond2 = n_train_empty > (len(train) - train_min) # if number of training samples is greater than specified minimum.

        n_values_total += n
        n_values_test += len(test)
        n_values_train += len(train)

        n_series_empty += n==n_empty
        n_test_empty_total+=n_test_empty
        n_train_empty_total+=n_train_empty
        n_series_test_empty += cond1
        n_series_train_min_empty += cond2

        if cond1 or cond2:
            n_series+=1
            n_values+=len(series[key])
            n_non_empty_values+=(len(series[key]) - n_empty)
            excluded_test_index.extend(test[test[target].isna()==False].index.tolist())

        else:
            n_values+=n_empty
            clean_series[key] = series[key].dropna(subset=['area'])
            if drop_start_zeros:
                test_msk = clean_series[key]['year'].isin(['01/01/2016', '01/01/2017'])
                train_msk = test_msk==False
                isNotZero = (clean_series[key][train_msk]['area']!=0).tolist()
                if sum(isNotZero)!=0:
                    testset = clean_series[key][test_msk]
                    trainset_cut = clean_series[key][train_msk].iloc[isNotZero.index(True):]
                    clean_series[key] = pd.concat([trainset_cut, testset])
                    n_train_zeros+=(sum(train_msk)-len(trainset_cut))


    print('Values Count:')
    print( pd.Series(
        [n_total, n_values_total, n_values_test, n_values_train],
        index=['n_total', 'n_values_total', 'n_values_test', 'n_values_train'])
    )
    print(); print('Excluded count:')
    print(
        pd.Series([
            n_series, n_series_empty, n_series_test_empty, n_series_train_min_empty,
            n_values, n_values-n_non_empty_values, n_test_empty_total, n_train_empty_total,
            n_non_empty_values, len(excluded_test_index), n_train_zeros
        ],
        index=[
            'series', 'series_empty', 'series_test_empty', f'series_train_min_<_{train_min}',
            'values', 'values_empty', 'test_empty', 'train_empty',
            'non_empty_values', 'non_empty_test_values', 'n_train_zeros'
        ])
    )
    return clean_series, excluded_test_index

def test_data():
    Y_e = []
    for key in series.keys():
        x_t, x_e, y_t, y_e = custom_tts(key)
        Y_e.append(y_e)
    return pd.concat(Y_e)

def get_prod_indexes():
    prod_indexes = {}
    for prod in prods:
        if prod not in ['Sorghum', 'Açaí']:
            if '-' in prod: 
                product, prod_type = prod.split('-')
            else:
                product = prod
            df = data_e[data_e['product']==product].copy()        
            if '-' in prod: 
                df = df[df['product_type']==prod_type]

            prod_indexes[prod] = df.index.copy()
    return prod_indexes

def get_prodtype_indexes():
    prodtype_indexes = {}
    for prodtype in ['temporary', 'permanent', 'pasture']:
        df = data_e[data_e['product_type']==prodtype].copy()        
        prodtype_indexes[prodtype] = df.index.copy()
    return prodtype_indexes

#### Mapping between the series test indexes and the series key
def get_key_index_test_map():    
    key_test_index_map = {}
    for key in series.keys():    
        key_index = series[key].index
        for entry_index in key_index:
            if entry_index in Y_e.index:
                key_test_index_map[entry_index] = key
    return key_test_index_map

def get_key_prodtype_map():
    key_prodtype_map = {}
    for key in series.keys():
        key_prodtype_map[key] = series[key].iloc[0]['product_type']
    return key_prodtype_map

#### Extract predictive and target variables
def get_xy(data, target='area', fill_na=0):
    ## 1. Extraindo variáveis preditivas
    X = data.drop(target, 1)
    ## 2. Extraindo Variável Álvo
    Y = data[target].copy()
    #### Substituição de Valores Vazios para Variável Alvo
#     Y[Y.isna()] = fill_na
    return X,Y

#### Label Encode pandas dataframe
def label_encode(X, base=None):
    if type(base)==type(None): base = X
    X_lab = X.copy()
    for column in X:
        X_lab[column] = le().fit(base[column]).transform(X[column])
    return X_lab

## Separação das amostras para treinamento e teste
def split_ts(x, y, col='year', horizon=2):
#     periods = x[col].sort_values().iloc[:len(x)-horizon]
    msk = x[col].isin([42, 43])
    return [x[msk==False], x[msk], y[msk==False], y[msk]]

### Train test split of time series by key
def custom_tts(key, encode=True):
    x, y = get_xy(series[key], target='area', fill_na=0)
    if encode: x = label_encode(x[['year']], base=data)
    return split_ts(x, y, col='year', horizon=2)

## Evaluating Model Predictions
def score_predictions(y_e, y_hat):
    return [mae(y_e, y_hat), mse(y_e, y_hat), mape(y_e, y_hat), wape(y_e, y_hat), r2(y_e, y_hat)]

## Evaluating model predictions given scorer list
def score_predictions_custom(y_e, y_hat, scorers=None):
    if type(scorers)==type(None):
        scorers = [mae, mse, mape, wape, r2]; names=scrs
    return [scorer(y_e, y_hat) for scorer in scorers]

## Evaluating model predictions given scorer list
def score_predictions_fancy(y_e, y_hat, model_name='', scorers=None, names=None):
    if type(scorers)==type(None):
        scorers = [mae, mse, mape, wape, r2]; names=scrs
    scores = [scorer(y_e, y_hat) for scorer in scorers]
    return pd.Series(scores, name=model_name, index=names)

def isComplete(ts):
    return (ts.index.max().year - ts.index.min().year + 1) == len(ts)

def train_test_min_max_year(train, test):
    return train.index.min().year, train.index.max().year, test.index.min().year, test.index.max().year

def train_test_min_max(train, test):
    return train.index.min(), train.index.max(), test.index.min(), test.index.max()


---
### 1. Loading data

In [4]:
target = 'area'
target_years = ['01/01/2016', '01/01/2017']
prodtypes = ['permanent', 'temporary', 'pasture']
prods = [
    'Rice', 'Beans', 'Cassava',
    'Corn', 'Soy', 'Sorghum',
    'Cocoa', 'Palm oil', 'Açaí',
    'Livestock', 'Others-temporary', 'Others-permanent'
]

In [5]:
data = pd.read_csv('data/historical-database-clean.csv')
raw_series = load_series()

Done! Loaded 1712 files.


---
## 2. Processing data

### 2.0 Dealing with missing values:

* Excluindo séries para as quais a variável alvo está vazia para 2016 e 2017.
* Excluindo séries para as quais a variável alvo tem menos que "n=1" entradas no conjunto de treinamento (35 found).
* Excluindo entradas vazias.

In [6]:
series, excluded_test_index = clean_series(raw_series, train_min=1)
keys = list(series.keys())

Values Count:
n_total            1712
n_values_total    73744
n_values_test      3424
n_values_train    70320
dtype: int64

Excluded count:
series                     605
series_empty               442
series_test_empty          582
series_train_min_<_1       465
values                   40936
values_empty             39035
test_empty                1198
train_empty              37837
non_empty_values          1901
non_empty_test_values       35
n_train_zeros             4796
dtype: int64


#### Concatening series

In [7]:
LE = le().fit(data['year'])
X_full = pd.concat([series[key].set_index('year')[target] for key in keys], 1).sort_index()
X_full.columns = keys; X_full.index = LE.transform(X_full.index.values)

In [42]:
X_train = X_full.loc[range(42)]
X_test = X_full.loc[[42, 43]]

#### Setting pandas datetime index
dt_series = {}
for key in keys:
    dt_series[key] = series[key].set_index(pd.DatetimeIndex(series[key]['year'].astype('datetime64[Y]')))[target]

#### Setting label encoded index

In [30]:
lab_series = {}
for key in keys:
    lab_series[key] = series[key].set_index(LE.transform(series[key]['year']))[target]

#### Settings train and test series

tt_series = {}
for key in keys:
    tt_series[key] = {'train': dt_series[key][:'2015'].copy(), 'test': dt_series[key]['2016':].copy()}

In [31]:
tt_lab_series = {}
for key in keys:
    msk = lab_series[key].index.isin([42, 43])
    tt_lab_series[key] = {'train': lab_series[key][msk==False].copy(), 'test': lab_series[key][msk].copy()}

#### Extracting full time series for testing

In [15]:
#### Extracting full series keys
full_keys = []
for key in keys:
    if len(tt_lab_series[key]['train'])==42 and len(tt_lab_series[key]['test'])==2:        
        full_keys.append(key)

#### Concating full time series
X = pd.concat([lab_series[key] for key in full_keys], 1); X.columns = full_keys
Xt = X.loc[:41]
Xe = X.loc[[42, 43]]

In [14]:
tt_lab_series[keys[0]]['train']

41    55.0
Name: area, dtype: float64

---
## Comparing specialized models

#### Metric criteria settings

In [15]:
criteria_map = {'e': 0, 'estd': 0, 'max_error': 0, 'mae': 0, 'mse': 0, 'medae': 0, 'mape': 0, 'wape': 0, 'r2': -1, 'evs': -1}
scorers_names = ['e','estd','max_error','mae','mse','medae','mape','wape','r2','evs']
scorers = [e, estd, me, mae, mse, medae, mape, wape, r2, evs]
scrs = ['mae', 'mse', 'mape', 'wape', 'r2']
sel_scrs = scorers_names #scrs

ts_models_scrs, models_scrs, models_prod_scrs, models_prediction = (models) => save_models=True, save_scores=True, scorers=scorers, scorers_names=scorers_names)

In [17]:
# contrived train and test datasets
train = tt_lab_series[full_keys[0]]['train']
test = tt_lab_series[full_keys[0]]['test']

# train and test prediction intervals
x_train_min, x_train_max, x_test_min, x_test_max = train_test_min_max(train, test)
# x_train_min_year, x_train_max_year, x_test_min_year, x_test_max_year = train_test_min_max_year(train, test)

In [18]:
x_train_min, x_train_max, x_test_min, x_test_max

(0, 41, 42, 43)

## AR example

In [ ]:
def AutoReg_predict(train, lags=1, x_min=42, x_max=43):
    if len(train)<=3: return np.array([np.nan for i in range(2)])
    model = AutoReg(train, lags=lags)
    model_fit = model.fit()
    # make prediction
    return model_fit.predict(x_min, x_max)

def AutoReg_predict_series(tt_series, keys, lags=1):
    yhat = {}
    for key in keys:
        co(wait=True); print(f'{keys.index(key)}/{len(keys)}')
        testset = tt_series[key]['test']
        trainset = tt_series[key]['train']
        yhat[key] = AutoReg_predict(trainset.values, lags=lags, x_min=len(trainset), x_max=len(trainset)+1)
        for i, ind in zip([0, 1], [42, 43]):
            if ind not in testset.index:
                yhat[key][i] = np.nan
                
    return pd.DataFrame(yhat)

In [ ]:
Yhat_AutoReg = AutoReg_predict_series(tt_lab_series, keys, lags=1)
yhat = Yhat_AutoReg.values.reshape(-1)

In [46]:
x_test = X_test.values.reshape(-1)
score_predictions_fancy(x_test, yhat)

## ARIMA example

In [ ]:
def arima_predict(train, order=(1,1,1), x_min=42, x_max=43):
    if len(train)<=2: return np.array([np.nan for i in range(2)])
    # fit model
    model = ARIMA(train, order=order)
    model_fit = model.fit()
    # make prediction
    return model_fit.predict(x_min, x_max)

def arima_predict_series(tt_series, keys, order=(1,1,1)):
    yhat_arima = {}
    for key in keys:
        co(wait=True); print(f'{keys.index(key)}/{len(keys)}')
        testset = tt_series[key]['test']
        trainset = tt_series[key]['train']
        yhat_arima[key] = arima_predict(trainset.values, order=order, x_min=len(trainset), x_max=len(trainset)+1)
        for i, ind in zip([0, 1], [42, 43]):
            if ind not in testset.index:
                yhat_arima[key][i] = np.nan
    
    return pd.DataFrame(yhat_arima, index=[42, 43])

In [72]:
Yhat_arima = arima_predict_series(tt_lab_series, keys, order=(1,1,1))

In [ ]:
score_predictions_fancy(x_test, Yhat_arima.values.reshape(-1))

## SARIMAX example

In [121]:
# contrived train and test datasets
def predict_score_SARIMAX(series, key, order=(1, 1, 1), seasonal_order=(0, 0, 0, 0), exog_train=None, exog_test=None):
    train = series[key]['train']
    test = series[key]['test']
    if len(train)<=2: return np.array([np.nan for i in range(2)])
    # fit model
    model = SARIMAX(train.values, exog=exog_train, order=order, seasonal_order=seasonal_order)
    model_fit = model.fit(disp=False)
    # make prediction
    yhat = model_fit.predict(len(train), len(train)+1, exog=exog_test)
    for i, ind in zip([0, 1], [42, 43]):
        if ind not in test.index:
            yhat[i] = np.nan
    return yhat

def predict_score_series_SARIMAX(series, keys):
    yhat = []
    for key in keys:
        co(wait=True); print(f'{keys.index(key)}/{len(keys)}')
        yhat_sarimax = predict_score_SARIMAX(tt_lab_series, key, order=(1, 1, 1), seasonal_order=(0,0,0,0))
        yhat.append(yhat_sarimax)
    yhat = pd.DataFrame(yhat).T; yhat.columns=keys
    return yhat

In [122]:
Yhat_sarimax = predict_score_series_SARIMAX(tt_lab_series, keys[:])

1106/1107


In [123]:
Yhat_sarimax

,0307883b5d063703-Açaí,0307883b5d063703-Cassava,0307883b5d063703-Livestock,0307883b5d063703-Others-permanent,0307883b5d063703-Others-temporary,03e477d4ede00e89-Beans,03e477d4ede00e89-Cassava,03e477d4ede00e89-Corn,03e477d4ede00e89-Livestock,03e477d4ede00e89-Others-permanent,03e477d4ede00e89-Others-temporary,03e477d4ede00e89-Rice,051e9cc7d7636816-Açaí,051e9cc7d7636816-Beans,051e9cc7d7636816-Cassava,051e9cc7d7636816-Cocoa,051e9cc7d7636816-Corn,051e9cc7d7636816-Livestock,051e9cc7d7636816-Others-permanent,051e9cc7d7636816-Others-temporary,051e9cc7d7636816-Rice,062ce4c17e84291e-Cassava,062ce4c17e84291e-Livestock,062ce4c17e84291e-Others-permanent,062ce4c17e84291e-Others-temporary,074590cfc1c41ebf-Açaí,074590cfc1c41ebf-Beans,074590cfc1c41ebf-Cassava,074590cfc1c41ebf-Cocoa,074590cfc1c41ebf-Corn,074590cfc1c41ebf-Livestock,074590cfc1c41ebf-Others-permanent,074590cfc1c41ebf-Others-temporary,074590cfc1c41ebf-Rice,074590cfc1c41ebf-Soy,07bba1a59af78f2d-Açaí,07bba1a59af78f2d-Beans,07bba1a59af78f2d-Cassava,07bba1a59af78f2d-Corn,07bba1a59af78f2d-Livestock,07bba1a59af78f2d-Others-permanent,07bba1a59af78f2d-Others-temporary,07bba1a59af78f2d-Palm oil,07bba1a59af78f2d-Rice,07ce2e79e19977e9-Açaí,07ce2e79e19977e9-Cassava,07ce2e79e19977e9-Corn,07ce2e79e19977e9-Livestock,07ce2e79e19977e9-Others-permanent,07ce2e79e19977e9-Others-temporary,07ed828db19fe8ca-Beans,07ed828db19fe8ca-Cassava,07ed828db19fe8ca-Corn,07ed828db19fe8ca-Livestock,07ed828db19fe8ca-Others-permanent,07ed828db19fe8ca-Others-temporary,07ed828db19fe8ca-Rice,09791645137aa877-Livestock,09791645137aa877-Others-permanent,09791645137aa877-Others-temporary,0dcbfaab30eb40c1-Açaí,0dcbfaab30eb40c1-Beans,0dcbfaab30eb40c1-Cassava,0dcbfaab30eb40c1-Corn,0dcbfaab30eb40c1-Livestock,0dcbfaab30eb40c1-Others-permanent,0dcbfaab30eb40c1-Others-temporary,0dcbfaab30eb40c1-Rice,0ea136556d12fc16-Beans,0ea136556d12fc16-Cassava,0ea136556d12fc16-Cocoa,0ea136556d12fc16-Corn,0ea136556d12fc16-Livestock,0ea136556d12fc16-Others-permanent,0ea136556d12fc16-Others-temporary,0ea136556d12fc16-Rice,1096f26c5a236d0b-Beans,1096f26c5a236d0b-Cassava,1096f26c5a236d0b-Corn,1096f26c5a236d0b-Livestock,1096f26c5a236d0b-Others-permanent,1096f26c5a236d0b-Others-temporary,10973d6049af898b-Açaí,10973d6049af898b-Beans,10973d6049af898b-Cassava,10973d6049af898b-Corn,10973d6049af898b-Livestock,10973d6049af898b-Others-permanent,10973d6049af898b-Others-temporary,10973d6049af898b-Rice,10973d6049af898b-Soy,114487007645c940-Açaí,114487007645c940-Beans,114487007645c940-Cassava,114487007645c940-Corn,114487007645c940-Livestock,114487007645c940-Others-permanent,114487007645c940-Others-temporary,114487007645c940-Rice,11645a57d3a1d507-Açaí,11645a57d3a1d507-Beans,11645a57d3a1d507-Cassava,11645a57d3a1d507-Cocoa,11645a57d3a1d507-Corn,11645a57d3a1d507-Livestock,11645a57d3a1d507-Others-permanent,11645a57d3a1d507-Others-temporary,11645a57d3a1d507-Palm oil,11b6622567f98f3b-Beans,11b6622567f98f3b-Cassava,11b6622567f98f3b-Corn,11b6622567f98f3b-Livestock,11b6622567f98f3b-Others-permanent,11b6622567f98f3b-Others-temporary,11b6622567f98f3b-Palm oil,13804a196311e1ba-Açaí,13804a196311e1ba-Beans,13804a196311e1ba-Cassava,13804a196311e1ba-Cocoa,13804a196311e1ba-Corn,13804a196311e1ba-Livestock,13804a196311e1ba-Others-permanent,13804a196311e1ba-Others-temporary,13804a196311e1ba-Rice,164c8953d3af2bbe-Açaí,164c8953d3af2bbe-Beans,164c8953d3af2bbe-Cassava,164c8953d3af2bbe-Cocoa,164c8953d3af2bbe-Corn,164c8953d3af2bbe-Livestock,164c8953d3af2bbe-Others-permanent,164c8953d3af2bbe-Others-temporary,164c8953d3af2bbe-Rice,18e37d24ff8517ef-Beans,18e37d24ff8517ef-Cassava,18e37d24ff8517ef-Corn,18e37d24ff8517ef-Livestock,18e37d24ff8517ef-Others-permanent,18e37d24ff8517ef-Others-temporary,18e37d24ff8517ef-Rice,18e37d24ff8517ef-Soy,18fb5197aa553b9c-Beans,18fb5197aa553b9c-Cassava,18fb5197aa553b9c-Corn,18fb5197aa553b9c-Livestock,18fb5197aa553b9c-Others-permanent,18fb5197aa553b9c-Others-temporary,1bd7145ed1da044a-Açaí,1bd7145ed1da044a-Beans,1bd7145ed1da044a-Cassava,1bd7145ed1da044a-Cocoa,1bd7

In [ ]:
score_predictions_fancy(X_test.values.reshape(-1), Yhat_sarimax.values.reshape(-1))

## VAR example

In [133]:
X_train.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41],
           dtype='int64')

In [129]:
def predict_VAR(X_train, steps=2, exog_train=None, exog_test=None):
    # fit model
    model = VAR(X_train, exog=exog_train)
    model_fit = model.fit()
    # make prediction
    yhat = model_fit.forecast(model_fit.y, steps=steps, exog_future=exog_test)
    return pd.DataFrame(yhat, columns=X_train.columns, index=range(len(X_train), len(X_train)+steps))

In [134]:
Yhat_var = predict_VAR(X_train.loc[14:].fillna(0), steps=2)

C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\base\wrapper.py:36: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


In [132]:
Yhat_var

,0307883b5d063703-Açaí,0307883b5d063703-Cassava,0307883b5d063703-Livestock,0307883b5d063703-Others-permanent,0307883b5d063703-Others-temporary,03e477d4ede00e89-Beans,03e477d4ede00e89-Cassava,03e477d4ede00e89-Corn,03e477d4ede00e89-Livestock,03e477d4ede00e89-Others-permanent,03e477d4ede00e89-Others-temporary,03e477d4ede00e89-Rice,051e9cc7d7636816-Açaí,051e9cc7d7636816-Beans,051e9cc7d7636816-Cassava,051e9cc7d7636816-Cocoa,051e9cc7d7636816-Corn,051e9cc7d7636816-Livestock,051e9cc7d7636816-Others-permanent,051e9cc7d7636816-Others-temporary,051e9cc7d7636816-Rice,062ce4c17e84291e-Cassava,062ce4c17e84291e-Livestock,062ce4c17e84291e-Others-permanent,062ce4c17e84291e-Others-temporary,074590cfc1c41ebf-Açaí,074590cfc1c41ebf-Beans,074590cfc1c41ebf-Cassava,074590cfc1c41ebf-Cocoa,074590cfc1c41ebf-Corn,074590cfc1c41ebf-Livestock,074590cfc1c41ebf-Others-permanent,074590cfc1c41ebf-Others-temporary,074590cfc1c41ebf-Rice,074590cfc1c41ebf-Soy,07bba1a59af78f2d-Açaí,07bba1a59af78f2d-Beans,07bba1a59af78f2d-Cassava,07bba1a59af78f2d-Corn,07bba1a59af78f2d-Livestock,07bba1a59af78f2d-Others-permanent,07bba1a59af78f2d-Others-temporary,07bba1a59af78f2d-Palm oil,07bba1a59af78f2d-Rice,07ce2e79e19977e9-Açaí,07ce2e79e19977e9-Cassava,07ce2e79e19977e9-Corn,07ce2e79e19977e9-Livestock,07ce2e79e19977e9-Others-permanent,07ce2e79e19977e9-Others-temporary,07ed828db19fe8ca-Beans,07ed828db19fe8ca-Cassava,07ed828db19fe8ca-Corn,07ed828db19fe8ca-Livestock,07ed828db19fe8ca-Others-permanent,07ed828db19fe8ca-Others-temporary,07ed828db19fe8ca-Rice,09791645137aa877-Livestock,09791645137aa877-Others-permanent,09791645137aa877-Others-temporary,0dcbfaab30eb40c1-Açaí,0dcbfaab30eb40c1-Beans,0dcbfaab30eb40c1-Cassava,0dcbfaab30eb40c1-Corn,0dcbfaab30eb40c1-Livestock,0dcbfaab30eb40c1-Others-permanent,0dcbfaab30eb40c1-Others-temporary,0dcbfaab30eb40c1-Rice,0ea136556d12fc16-Beans,0ea136556d12fc16-Cassava,0ea136556d12fc16-Cocoa,0ea136556d12fc16-Corn,0ea136556d12fc16-Livestock,0ea136556d12fc16-Others-permanent,0ea136556d12fc16-Others-temporary,0ea136556d12fc16-Rice,1096f26c5a236d0b-Beans,1096f26c5a236d0b-Cassava,1096f26c5a236d0b-Corn,1096f26c5a236d0b-Livestock,1096f26c5a236d0b-Others-permanent,1096f26c5a236d0b-Others-temporary,10973d6049af898b-Açaí,10973d6049af898b-Beans,10973d6049af898b-Cassava,10973d6049af898b-Corn,10973d6049af898b-Livestock,10973d6049af898b-Others-permanent,10973d6049af898b-Others-temporary,10973d6049af898b-Rice,10973d6049af898b-Soy,114487007645c940-Açaí,114487007645c940-Beans,114487007645c940-Cassava,114487007645c940-Corn,114487007645c940-Livestock,114487007645c940-Others-permanent,114487007645c940-Others-temporary,114487007645c940-Rice,11645a57d3a1d507-Açaí,11645a57d3a1d507-Beans,11645a57d3a1d507-Cassava,11645a57d3a1d507-Cocoa,11645a57d3a1d507-Corn,11645a57d3a1d507-Livestock,11645a57d3a1d507-Others-permanent,11645a57d3a1d507-Others-temporary,11645a57d3a1d507-Palm oil,11b6622567f98f3b-Beans,11b6622567f98f3b-Cassava,11b6622567f98f3b-Corn,11b6622567f98f3b-Livestock,11b6622567f98f3b-Others-permanent,11b6622567f98f3b-Others-temporary,11b6622567f98f3b-Palm oil,13804a196311e1ba-Açaí,13804a196311e1ba-Beans,13804a196311e1ba-Cassava,13804a196311e1ba-Cocoa,13804a196311e1ba-Corn,13804a196311e1ba-Livestock,13804a196311e1ba-Others-permanent,13804a196311e1ba-Others-temporary,13804a196311e1ba-Rice,164c8953d3af2bbe-Açaí,164c8953d3af2bbe-Beans,164c8953d3af2bbe-Cassava,164c8953d3af2bbe-Cocoa,164c8953d3af2bbe-Corn,164c8953d3af2bbe-Livestock,164c8953d3af2bbe-Others-permanent,164c8953d3af2bbe-Others-temporary,164c8953d3af2bbe-Rice,18e37d24ff8517ef-Beans,18e37d24ff8517ef-Cassava,18e37d24ff8517ef-Corn,18e37d24ff8517ef-Livestock,18e37d24ff8517ef-Others-permanent,18e37d24ff8517ef-Others-temporary,18e37d24ff8517ef-Rice,18e37d24ff8517ef-Soy,18fb5197aa553b9c-Beans,18fb5197aa553b9c-Cassava,18fb5197aa553b9c-Corn,18fb5197aa553b9c-Livestock,18fb5197aa553b9c-Others-permanent,18fb5197aa553b9c-Others-temporary,1bd7145ed1da044a-Açaí,1bd7145ed1da044a-Beans,1bd7145ed1da044a-Cassava,1bd7145ed1da044a-Cocoa,1bd7

In [144]:
not_na_msk_test = pd.Series(x_test).isna()==False
x_test_f = x_test[not_na_msk_test]

In [145]:
yhat_var = Yhat_var.values.reshape(-1)[not_na_msk_test]

In [146]:
score_predictions_fancy(x_test_f, yhat_var)

mae     1.567370e+03
mse     2.367336e+07
mape    1.137444e+16
wape    7.690203e-02
r2      9.969447e-01
Name: , dtype: float64

In [147]:
len(x_test_f)

2191

## VARMA example (not working for all series  at once)

In [36]:
# fit model
def predict_VARMAX(X_train, steps=2, order=(1, 1)):
    model = VARMAX(X_train, order=order)
    model_fit = model.fit(disp=False)
    # make prediction
    return model_fit.forecast(steps=steps)

In [39]:
Yhat_varma = predict_VARMAX(X_train.iloc[:, :2], steps=2)

C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


In [42]:
yhat = Yhat_varma.values.reshape(-1)
x_test = X_test.iloc[:, :2].values.reshape(-1)

In [43]:
score_predictions(x_test, yhat)

[13.14127815762053,
 311.53917647104606,
 0.26099213000069693,
 0.30739831947650365,
 0.6276516901817631]

## VARMAX example

In [ ]:
# fit model
model = VARMAX(X_train.iloc[:, 1:5], exog=X_train.iloc[:, 6:10], order=(1, 1))
model_fit = model.fit(disp=False)

In [1036]:
# make prediction
Yhat_varmax = model_fit.forecast(exog=X_test.iloc[:, 6:10], steps=2)
Yhat_varmax

C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\luisr\anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


,0307883b5d063703-Others-temporary,03e477d4ede00e89-Others-permanent,03e477d4ede00e89-Others-temporary,051e9cc7d7636816-Others-permanent
42,6.485532,506.906915,37.889265,15.258532
43,3.001125,504.499522,-11.166915,14.415549


In [853]:
score_predictions_custom(Yhat_varmax.values.reshape(-1), X_test.iloc[:, 1:5].values.reshape(-1))

[45.956374751841096,
 7202.773487870832,
 0.9593998911077435,
 0.3412740858180361,
 0.8436123022543823]

## AutoArima Example

In [ ]:
def predict_series_auto_arima(series, keys, n_periods=2):
    Yhat, scores = [], []
    for key in keys:
        co(wait=True); print(f'{keys.index(key)}/{len(keys)}')
        train = series[key]['train']
        test = series[key]['test']
        if len(train)<=2:
            Yhat.append(pd.Series([np.nan for i in range(2)], index=[42,43], name=key))
            continue
        model = auto_arima(y=train.values, seasonal=False, stepwise=True)
        yhat = model.predict(n_periods=n_periods)
        for i, ind in zip([0, 1], [42, 43]):
            if ind not in test.index:
                yhat[i] = np.nan
        Yhat.append(pd.Series(yhat, index=[42,43], name=key))
#         scores.append(score_predictions_fancy(test, yhat, model_name=key))
    return pd.concat(Yhat, 1)#, pd.concat(scores, 1)

In [177]:
Yhat_autoarima =  predict_series_auto_arima(tt_lab_series, keys)

1106/1107


In [ ]:
score_predictions_fancy(Yhat_autoarima.values.reshape(-1), X_test.values.reshape(-1))

### TBATS Example

In [56]:
def predict_tbats(train, steps):
    model_tbats = TBATS(seasonal_periods=(12, 28),\
                  use_arma_errors=False,\
                  use_box_cox=False,\
                  n_jobs=1,\
                  use_trend=None,\
                  use_damped_trend=None)\
                  .fit(train)

    return model_tbats.forecast(steps=steps)

In [57]:
def predict_series_tbats(series, keys, steps=2):
    Yhat, scores = [], []
    for key in keys:
        co(wait=True); print(f'{keys.index(key)}/{len(keys)}')
        train = series[key]['train']
        test = series[key]['test']
        yhat = predict_tbats(train, steps=len(test))
        Yhat.append(pd.Series(yhat, index=range(len(train), len(train)+steps), name=key))
        scores.append(score_predictions_fancy(test, yhat, model_name=key))
    return pd.concat(Yhat, 1), pd.concat(scores, 1)

In [62]:
Yhat_tbats, scores = predict_series_tbats(tt_lab_series, full_keys[:], steps=2)

285/286


In [63]:
score_predictions_fancy(X_test.iloc[:, :].values.reshape(-1), Yhat_tbats.values.reshape(-1))

mae     4.257729e+02
mse     1.108583e+06
mape    1.397161e+17
wape    8.801666e-01
r2      4.817428e-01
Name: , dtype: float64

---
## Comparing models

In [179]:
model_names = ['Yhat_AutoReg', 'Yhat_arima', 'Yhat_sarimax', 'Yhat_var', 'Yhat_autoarima']
model_predictions = [Yhat_AutoReg, Yhat_arima, Yhat_sarimax, Yhat_var, Yhat_autoarima]

In [187]:
y_hat_i = pd.concat([pd.Series(model.values.reshape(-1), name=model_names[i]) for i, model in enumerate(model_predictions)], 1)
y_hat_i.index = keys*2
y_hat_i['year'] = ['01/01/2016']*len(keys) + ['01/01/2017']*len(keys)
y_hat_i = y_hat_i.reset_index(drop=False).rename(columns={'index': 'key'})

In [192]:
#y_hat_i.to_csv('evaluation/specialized_models_predictions.csv', index=True)

In [70]:
x_test = X_test.values.reshape(-1)
models_scrs = pd.concat([score_predictions_fancy(x_test, y_hat_i[model], model_name=model) for model in y_hat_i], 1)

In [72]:
models_scrs

,Yhat_AutoReg,Yhat_arima,Yhat_sarimax,Yhat_var,Yhat_autoarima,Yhat_tbats
mae,1.150449e+02,1.042907e+02,1.042907e+02,1.293528e+13,1.098114e+02,4.257729e+02
mse,1.168531e+05,1.108632e+05,1.108632e+05,1.031415e+27,1.157460e+05,1.108583e+06
mape,1.643048e+16,9.369222e+15,9.369222e+15,5.444631e+27,9.376776e+15,1.397161e+17
wape,2.378232e-01,2.155918e-01,2.155918e-01,2.674009e+10,2.270043e-01,8.801666e-01
r2,9.453717e-01,9.481720e-01,9.481720e-01,-4.821817e+20,9.458893e-01,4.817428e-01


In [927]:
def plot_preds(Yhat, keys):
    fig = plt.figure(figsize=(12, 10), tight_layout=True)
    n_rows = len(keys)//3
    axes = fig.subplots(n_rows,3)
    cnt = -1
    for axs in axes:
        for i in range(len(axs)):
            if cnt<len(keys)-1:
                ax = axs[i]
                cnt+=1; key=keys[cnt]
                train = tt_lab_series[key]['train']
                test = tt_lab_series[key]['test']
                yhat = Yhat[key]
                train.plot(ax=ax)
                test.plot(ax=ax)
                yhat.plot(ax=ax)
    plt.show()